<a href="https://colab.research.google.com/github/rdemarqui/Data-Science-Basico/blob/master/Financial_distress_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Financial Distress Prediction

Source: https://www.kaggle.com/competitions/GiveMeSomeCredit/

In [44]:
import os
import zipfile
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score

In [52]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
# Getting current work dir
current_dir = os.getcwd()

In [6]:
# Downloading file
!wget -q https://github.com/rdemarqui/financial_distress_prediction/raw/main/datasets/GiveMeSomeCredit.zip

In [10]:
# Opening datasets
zf = zipfile.ZipFile(os.path.join(current_dir, 'GiveMeSomeCredit.zip'))
df_train = pd.read_csv(zf.open('cs-training.csv'), index_col=0, low_memory=False)
df_test = pd.read_csv(zf.open('cs-test.csv'), index_col=0, low_memory=False)

## Exploratory Data Analysis (EDA)

In [15]:
# Checking null values and data types
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

Dataset has only numeric features. Also has null values on *MonthlyIncome* and *NumberOfDependents*. I'll deal with that later.

In [13]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
SeriousDlqin2yrs,150000.0,0.066840,0.249746,0.0,0.000000,0.000000,0.000000,1.0
RevolvingUtilizationOfUnsecuredLines,150000.0,6.048438,249.755371,0.0,0.029867,0.154181,0.559046,50708.0
age,150000.0,52.295207,14.771866,0.0,41.000000,52.000000,63.000000,109.0
NumberOfTime30-59DaysPastDueNotWorse,150000.0,0.421033,4.192781,0.0,0.000000,0.000000,0.000000,98.0
DebtRatio,150000.0,353.005076,2037.818523,0.0,0.175074,0.366508,0.868254,329664.0
MonthlyIncome,120269.0,6670.221237,14384.674215,0.0,3400.000000,5400.000000,8249.000000,3008750.0
NumberOfOpenCreditLinesAndLoans,150000.0,8.452760,5.145951,0.0,5.000000,8.000000,11.000000,58.0
NumberOfTimes90DaysLate,150000.0,0.265973,4.169304,0.0,0.000000,0.000000,0.000000,98.0
NumberRealEstateLoansOrLines,150000.0,1.018240,1.129771,0.0,0.000000,1.000000,2.000000,54.0
NumberOfTime60-89DaysPastDueNotWorse,150000.0,0.240387,4.155179,0.0,0.000000,0.000000,0.000000,98.0


In [21]:
df_train

#df_train_final = df_train.copy()
#df_test_final = df_test.copy()

In [20]:
# Splitting data
X_train = df_train_final.drop(columns=['SeriousDlqin2yrs'])
y_train = df_train_final['SeriousDlqin2yrs']

In [ ]:
# Calculating target proportion
label_prop = round(np.sum(y_train==0)/np.sum(y_train==1), 2)
label_prop

##Training

In [31]:
# kflod split
nsplit = 5

13.96

### Baseline Model

In [57]:
# Baseline Model
kfold = StratifiedKFold(n_splits=nsplit, shuffle=True, random_state=42)
model = XGBClassifier(nthread=4)
scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')

print('roc_auc: ', np.round(scores, 2))
print('roc_auc mean: ', np.round(scores.mean(), 5))

roc_auc:  [0.85 0.86 0.86 0.85 0.86]
roc_auc mean:  0.85842


In [56]:
# Baseline Model
kfold = StratifiedKFold(n_splits=nsplit, shuffle=True, random_state=42)
model = LGBMClassifier()
scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')

print('roc_auc: ', np.round(scores, 2))
print('roc_auc mean: ', np.round(scores.mean(), 5))

roc_auc:  [0.86 0.86 0.87 0.86 0.87]
roc_auc mean:  0.86483


## Hiperparameter Tunning

## 